In [7]:
# Imports 
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import pandas_profiling.profile_report as pr

from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing 

In [16]:
# Encode Columns 
def lable_encoder(df, columns):
    Encoder = dict()

    for var in catergorical:
        Encoder[var] = preprocessing.LabelEncoder().fit(df[var].astype(str))
        
    return Encoder

def encode_columns(df, columns, encoder = None):
    if not encoder:
        encoder = lable_encoder(df, columns)
 
    for var in columns: 
        df[var] = encoder[var].transform(df[var].astype(str))
        
    return encoder

# Binarize Columns 
def label_bin(df, columns):
    Encoder = dict()

    for var in columns:
        Encoder[var] = preprocessing.LabelBinarizer().fit(df[var].astype(str))
        
    return Encoder

def bin_columns(df, columns, encoder = None):
    if not encoder:
        encoder = label_bin(df, columns)
    
    for var in columns: 
        df[var] = encoder[var].transform(df[var].astype(str))

    return encoder

In [48]:
# Import data 
df = pd.read_csv('../data/train_1.csv')

/home/noel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (11,13,39,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [49]:
catergorical = ['UsageBand', 'fiModelDesc', 'fiBaseModel', 'fiProductClassDesc', 'fiSecondaryDesc', 
                'fiModelSeries', 'fiModelDescriptor', 'ProductSize', 'ProductGroupDesc', 'Drive_System', 
                'Enclosure', 'Pad_Type', 'Transmission','Blade_Width', 'Enclosure_Type', 'Hydraulics',
                'state', 'ProductGroup', 'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
                'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type','Undercarriage_Pad_Width', 'Stick_Length', 
                'Thumb', 'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
                'Differential_Type', 'Steering_Controls', 'Scarifier']

# Binary 
binary = ['Forks', 'Ride_Control', 'Stick', 'Turbocharged', 'Blade_Extension', 'Engine_Horsepower', 
          'Pushblock', 'Ripper']

In [50]:
# ['MachineHoursCurrentMeter'] Missing 
df['MachineHoursCurrentMeter'] = df['MachineHoursCurrentMeter'].fillna(0)

In [51]:
# Encode data 
label_encoder = encode_columns(df, catergorical)
binary_encoder = bin_columns(df, binary)

In [52]:
# Change slatedate to year, month, day 
df['saleyear'] = pd.DatetimeIndex(df['saledate']).year
df['salemonth'] = pd.DatetimeIndex(df['saledate']).month
df['saleday'] = pd.DatetimeIndex(df['saledate']).day

In [53]:
# Drop variables 
df.drop(['SalesID','MachineID', 'ModelID', 'saledate'], axis=1, inplace = True)

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201832 entries, 0 to 201831
Data columns (total 52 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalePrice                 201832 non-null  int64  
 1   datasource                201832 non-null  int64  
 2   auctioneerID              201832 non-null  int64  
 3   YearMade                  201832 non-null  int64  
 4   MachineHoursCurrentMeter  201832 non-null  float64
 5   UsageBand                 201832 non-null  int64  
 6   fiModelDesc               201832 non-null  int64  
 7   fiBaseModel               201832 non-null  int64  
 8   fiSecondaryDesc           201832 non-null  int64  
 9   fiModelSeries             201832 non-null  int64  
 10  fiModelDescriptor         201832 non-null  int64  
 11  ProductSize               201832 non-null  int64  
 12  fiProductClassDesc        201832 non-null  int64  
 13  state                     201832 non-null  i

In [55]:
import statsmodels.api as sm

def summary_model(X, y, label='scatter'):
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    summary = model.summary()
    return summary

def plot_model(X, y, label='Residual Plot'):
    model = sm.OLS(y, X).fit()
    student_resids = model.outlier_test()['student_resid']
    y_hats = model.predict(X)

    plt.scatter(y_hats, student_resids, alpha = .35, label=label)
    plt.legend()
    plt.show()

In [56]:
X = df.drop('SalePrice', axis= 1)
y = df['SalePrice']

In [58]:
summary_model(X, y)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.376
Model:                            OLS   Adj. R-squared:                  0.376
Method:                 Least Squares   F-statistic:                     2383.
Date:                Fri, 05 Mar 2021   Prob (F-statistic):               0.00
Time:                        15:04:38   Log-Likelihood:            -2.2678e+06
No. Observations:              201832   AIC:                         4.536e+06
Df Residuals:                  201780   BIC:                         4.536e+06
Df Model:                          51                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                     5.995e+05   1.86e+04     32.319      0.000    5.63e+05    6.36e+05
datasource                 -25.8424     19.530     -1.323      0.186     -64.120      12.435
auctioneerID               -45.9372      2.533    -18.137      0.000     -50.901     -40.973
YearMade                    12.8241      0.230     55.669      0.000      12.373      13.276
MachineHoursCurrentMeter     0.0066      0.020      0.328      0.743      -0.033       0.046
UsageBand                -2710.5042     99.363    -27.279      0.000   -2905.253   -2515.755
fiModelDesc                  6.2391      0.620     10.062      0.000       5.024       7.454
fiBaseModel                -25.5487      1.726    -14.805      0.000     -28.931     -22.166
fiSecondaryDesc            -78.4721      1.205    -65.105      0.000     -80.834     -76.110
fiModelSeries             -244.6054      7.967    -30.703      0.000    -260.220    -228.990
fiModelDescriptor         -189.4263      2.594    -73.038      0.000    -194.510    -184.343
ProductSize              -2799.2318     31.265    -89.532      0.000   -2860.511   -2737.953
fiProductClassDesc        -503.4588     11.402    -44.157      0.000    -525.806    -481.112
state                       -3.7076      2.680     -1.383      0.167      -8.961       1.546
ProductGroup              2247.1242   1601.855      1.403      0.161    -892.472    5386.720
ProductGroupDesc         -1488.8186   2034.068     -0.732      0.464   -5475.542    2497.905
Drive_System             -2745.3904     89.702    -30.606      0.000   -2921.204   -2569.576
Enclosure                 -511.3878     23.597    -21.672      0.000    -557.637    -465.138
Forks                    -3577.0384    240.952    -14.845      0.000   -4049.299   -3104.778
Pad_Type                  2519.7357    187.050     13.471      0.000    2153.123    2886.349
Ride_Control              -1.44e+04    718.673    -20.037      0.000   -1.58e+04    -1.3e+04
Stick                      834.0581    187.255      4.454      0.000     467.044    1201.073
Transmission               342.3676     76.667      4.466      0.000     192.102     492.633
Turbocharged               116.5534    377.305      0.309      0.757    -622.955     856.062
Blade_Extension           2.057e+04    929.008     22.139      0.000    1.87e+04    2.24e+04
Blade_Width               1165.5070     79.928     14.582      0.000    1008.850    1322.164
Enclosure_Type            -2.44e+04    359.113    -67.949      0.000   -2.51e+04   -2.37e+04
Engine_Horsepower        -1.584e+04    747.882    -21.181      0.000   -1.73e+04   -1.44e+04
Hydraulics                 400.5520     25.229     15.877      0.000     351.103     450.001
Pushblock                -1.602e+04    367.449    -43.599      0.000   -1.67e+04   -1.53e+04
Ripper        